In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import random
import sys
import chromedriver_autoinstaller
import zipfile

/Users/solalzana/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()

tp_engie = "https://fr.trustpilot.com/review/engie.fr"
tp_edf = "https://fr.trustpilot.com/review/edf.fr"
tp_total = "https://fr.trustpilot.com/review/totalenergies.fr"
tp_octopus = "https://fr.trustpilot.com/review/octopusenergy.fr"
tp_plenitude = "https://fr.trustpilot.com/review/eniplenitude.fr"
tp_ekwateur = "https://fr.trustpilot.com/review/ekwateur.fr"
tp_mint = "https://fr.trustpilot.com/review/www.mint-energie.com"
tp_alpiq = "https://fr.trustpilot.com/review/particuliers.alpiq.fr"
tp_ilek = "https://fr.trustpilot.com/review/ilek.fr"
tp_ohm = "https://fr.trustpilot.com/review/ohm-energie.com"
tp_elmy = "https://fr.trustpilot.com/review/elmy-energie.fr"
tp_vattenfall = "https://fr.trustpilot.com/review/vattenfall.fr"
tp_clicandfioul = "https://fr.trustpilot.com/review/www.clicandfioul.com"

driver = webdriver.Chrome(options=chrome_options)
driver.get(tp_engie)
# driver.get(tp_edf)
# driver.get(tp_total)
# driver.get(tp_octopus)
# driver.get(tp_plenitude)
# driver.get(tp_ekwateur)
# driver.get(tp_mint)
# driver.get(tp_engie)
# driver.get(tp_alpiq)
# driver.get(tp_ilek)
# driver.get(tp_ohm)
# driver.get(tp_elmy)
# driver.get(tp_vattenfall)
# driver.get(tp_clicandfioul)

In [3]:
cookie_button = driver.find_element(By.XPATH, "//*[@id='onetrust-accept-btn-handler']")
cookie_button.click()

### Proxy handling for ip:port:user:pw format

In [4]:
def create_proxy_extension(proxy_host, proxy_port, username, password):
    """
    Create a Chrome extension for handling proxy authentication.
    """
    manifest_json = """{
        "version": "1.0.0",
        "manifest_version": 2,
        "name": "Chrome Proxy",
        "permissions": [
            "proxy",
            "tabs",
            "unlimitedStorage",
            "storage",
            "<all_urls>",
            "webRequest",
            "webRequestBlocking"
        ],
        "background": {
            "scripts": ["background.js"]
        },
        "minimum_chrome_version":"22.0.0"
    }"""

    background_js = f"""
    var config = {{
            mode: "fixed_servers",
            rules: {{
            singleProxy: {{
                scheme: "http",
                host: "{proxy_host}",
                port: parseInt({proxy_port})
            }},
            bypassList: ["localhost"]
            }}
        }};

    chrome.proxy.settings.set({{value: config, scope: "regular"}}, function() {{}});
    
    chrome.webRequest.onAuthRequired.addListener(
        function(details) {{
            return {{
                authCredentials: {{
                    username: "{username}",
                    password: "{password}"
                }}
            }};
        }},
        {{urls: ["<all_urls>"]}},
        ["blocking"]
    );
    """

    # Create zip extension
    plugin_file = "proxy_auth_plugin.zip"
    with zipfile.ZipFile(plugin_file, "w") as zp:
        zp.writestr("manifest.json", manifest_json)
        zp.writestr("background.js", background_js)
    
    return plugin_file

In [5]:
def load_proxies(file_path):
    with open(file_path, "r") as file:
        return [line.strip().split(":") for line in file if line.strip()]

def get_chrome_driver(proxy_host, proxy_port, username, password):
    plugin_file = create_proxy_extension(proxy_host, proxy_port, username, password)
    
    chrome_options = Options()
    chrome_options.add_extension(plugin_file)  
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    return webdriver.Chrome(options=chrome_options)

proxy_list = load_proxies("proxies.txt")

urls = [
    "https://fr.trustpilot.com/review/engie.fr",
    "https://fr.trustpilot.com/review/edf.fr",
    "https://fr.trustpilot.com/review/totalenergies.fr"
]

reviews = []


for url in urls:
    # Select a random proxy
    print(f"\n Using Proxy: {proxy_host}:{proxy_port} for {url}")

    driver = get_chrome_driver(proxy_host, proxy_port, username, password)
    driver.get(url)
    time.sleep(random.uniform(0.2, 0.8))  # Random delay

    total_pages = int(driver.find_element(By.XPATH, "//*[@id='__next']/div/div/main/div/div[4]/section/div[26]/nav/a[4]/span").text)

    for page in range(1, total_pages + 1):
        proxy_host, proxy_port, username, password = random.choice(proxy_list)
        print(f"Scraping page {page} of {total_pages}")

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/div/div/main/div/div[4]/section/div/article/div/section"))
        )

        time.sleep(random.uniform(0.2, 0.8))

        comments = driver.find_elements(By.XPATH, "//*[@id='__next']/div/div/main/div/div[4]/section/div/article/div/section")

        for i, comment in enumerate(comments, start=1):
            try:
                title = comment.find_element(By.XPATH, "./div[2]/a/h2").text if comment.find_elements(By.XPATH, "./div[2]/a/h2") else "No title"
                body = comment.find_element(By.XPATH, "./div[2]/p[1]").text if comment.find_elements(By.XPATH, "./div[2]/p[1]") else "No body"
                date = comment.find_element(By.XPATH, "./div[1]/div[2]/time").text if comment.find_elements(By.XPATH, "./div[1]/div[2]/time") else "No date"
                note_element = comment.find_elements(By.XPATH, "./div[1]/div[1]/img")

                note = note_element[0].get_attribute("alt")[5] if note_element else "No rating"

                reviews.append({
                    "url": url,
                    "proxy": proxy_host,
                    "comment": f"comment {i} (page {page})",
                    "title": title,
                    "body": body,
                    "date": date,
                    "note": note
                })

            except Exception as e:
                print(f"Error processing comment {i} on page {page}: {e}")

        if page < total_pages:
            try:
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//*[@id='__next']/div/div/main/div/div[4]/section/div[26]/nav/a[5]"))
                )

                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                driver.execute_script("arguments[0].click();", next_page_button)
                time.sleep(random.uniform(0.2, 0.8))

            except Exception as e:
                print(f"No more pages or error: {e}")
                break

    driver.quit()

print(f"\nTotal Reviews Extracted: {len(reviews)}")


🔄 Using Proxy: gw-eu.lemonclub.io:5555 for https://fr.trustpilot.com/review/engie.fr
Scraping page 1 of 231
Scraping page 2 of 231
Scraping page 3 of 231
Scraping page 4 of 231
Scraping page 5 of 231
Scraping page 6 of 231
Scraping page 7 of 231
Scraping page 8 of 231


KeyboardInterrupt: 

In [ ]:
reviews = pd.DataFrame(reviews)
reviews

In [7]:
reviews.to_csv("/Users/solalzana/Desktop/X/S2/Capgemini/Lesson 1/reviews_trustpilot.csv")